In [50]:
import serial
import serial.tools.list_ports
import time



def setPosition(x,y,z):
    serialDelta.write(bytes('G1 X' + str(x) + ' Y' + str(y) + ' Z' + str(z) + ' F6000\n', 'utf8'))

serialRomeo = ''
serialDelta = ''

ports = list(serial.tools.list_ports.comports())
for p in ports:
    print(p[2])
    if 'USB VID:PID=2341:0043' in p[2]:
        try:
            serialRomeo = serialBarcode = serial.Serial(p[0], 115200, timeout=0.5)
            print("Romeo connected!")
        except:
            print("Romeo not connected!")
    if 'USB VID:PID=2341:0042' in p[2]:
        try:
            serialDelta = serialBarcode = serial.Serial(p[0], 250000, timeout=0.5)
            print("Delta connected!")
        except:
            print("Delta not connected!")

            
time.sleep(2)
            
serialDelta.write(b'G28\n')


# serialRomeo.write(b'<speed>0,0;')

setPosition(0,0,150)

# time.sleep(1)

# serialRomeo.close()
# serialDelta.close()

n/a
n/a
n/a
n/a
n/a
n/a
USB VID:PID=2341:0043 LOCATION=20-5.1.2
Romeo connected!
USB VID:PID=2341:0042 SER=75430333237351E012A2 LOCATION=20-5.1.1.3
Delta connected!


False

In [ ]:
# serialDelta.write(b'G1 Z200 F4000\n')

In [ ]:
import cv2
from PIL import Image
import threading
from http.server import BaseHTTPRequestHandler,HTTPServer
from socketserver import ThreadingMixIn
from io import StringIO,BytesIO
import time
capture=None

from collections import deque
import numpy as np
import argparse


delta = [0,0,150]

# define the lower and upper boundaries of the colors in the HSV color space
lower = {'red':(166, 84, 141), 'yellow':(23, 59, 119)} #assign new item lower['blue'] = (93, 10, 0)
upper = {'red':(186,255,255), 'yellow':(54,255,255)}

# define standard colors for circle around the object
colors = {'red':(0,0,255), 'yellow':(0, 255, 217)}        

class CamHandler(BaseHTTPRequestHandler):
    def do_GET(self):
        if self.path.endswith('.mjpg'):
            self.send_response(200)
            self.send_header('Content-type','multipart/x-mixed-replace; boundary=--jpgboundary')
            self.end_headers()
            while True:
                try:
                    rc,img = capture.read()
                    if not rc:
                        continue
                    blurred = cv2.GaussianBlur(img, (11, 11), 0)
                    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
                    #for each color in dictionary check object in frame
                    for key, value in upper.items():
                        # construct a mask for the color from dictionary`1, then perform
                        # a series of dilations and erosions to remove any small
                        # blobs left in the mask
                        kernel = np.ones((9,9),np.uint8)
                        mask = cv2.inRange(hsv, lower[key], upper[key])
                        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
                        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

                        # find contours in the mask and initialize the current
                        # (x, y) center of the ball
                        cnts = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL,
                            cv2.CHAIN_APPROX_SIMPLE)[-2]
                        center = None

                        # only proceed if at least one contour was found
                        if len(cnts) > 0:
                            # find the largest contour in the mask, then use
                            # it to compute the minimum enclosing circle and
                            # centroid
                            
                            for c in cnts:
                                ((x, y), radius) = cv2.minEnclosingCircle(c)
                                M = cv2.moments(c)
                                center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))

                                # only proceed if the radius meets a minimum size. Correct this value for your obect's size
                                if radius > 0.5:
    
                                    # draw the circle and centroid on the frame,
                                    # then update the list of tracked points
                                    cv2.rectangle(img, (int(x-radius/2), int(y-radius/2)), (int(x+radius/2), int(y+radius/2)), (93, 10, 0), 2)
                                    cv2.putText(img,key + " grass", (int(x-radius),int(y-radius)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (93, 10, 0),2)


                            
                            c = max(cnts, key=cv2.contourArea)
                            ((x, y), radius) = cv2.minEnclosingCircle(c)
                            M = cv2.moments(c)
                            center = (int(M["m10"] / M["m00"]), int(M["m01"] / M["m00"]))
                            

                            # only proceed if the radius meets a minimum size. Correct this value for your obect's size
                            if radius > 0.5:
                                newDeltaX = delta[0] - (y - 300.0)/60.0
                                newDeltaY = delta[1] + (x - 400.0)/80.0

                            
                                if (newDeltaX*newDeltaX + newDeltaY*newDeltaY < 3600):                            
                                    delta[0] = newDeltaX
                                    delta[1] = newDeltaY
                                    setPosition(delta[0],delta[1],delta[2])


                                # draw the circle and centroid on the frame,
                                # then update the list of tracked points
                                cv2.rectangle(img, (int(x-radius/2), int(y-radius/2)), (int(x+radius/2), int(y+radius/2)), (186,255,255), 2)
                                cv2.putText(img,key + " grass", (int(x-radius),int(y-radius)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (186,255,255),2)
                    imgRGB=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                    jpg = Image.fromarray(imgRGB)
                    tmpFile = BytesIO()
                    jpg.save(tmpFile,'JPEG')
                    self.wfile.write("--jpgboundary".encode())
                    self.send_header('Content-type','image/jpeg')
                    self.send_header('Content-length',str(tmpFile.getbuffer().nbytes))
                    self.end_headers()
                    jpg.save(self.wfile,'JPEG')
#                     time.sleep(0.05)
                except KeyboardInterrupt:
                    print("stop streaming")
                    break
#                 except:
#                     print("other error streaming")
#                     pass
            return
        else:
            self.send_response(200)
            self.send_header('Content-type','text/html')
            self.end_headers()
            self.wfile.write('<html><head></head><body>'.encode())
            self.wfile.write('<img src="http://127.0.0.1:9000/cam.mjpg"/>'.encode())
            self.wfile.write('</body></html>'.encode())
            return


capture = cv2.VideoCapture(1)
time.sleep(1)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 800); 
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 600);
capture.set(cv2.CAP_PROP_SATURATION,0.2);
server = HTTPServer(('localhost', 9000), CamHandler)
print( "server started")
try:
    server.serve_forever()
except KeyboardInterrupt:
    print("stop server")
    pass
except:
    print("other error server")
    pass
server.server_close()
capture.release()



server started


127.0.0.1 - - [19/Aug/2018 21:11:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Aug/2018 21:11:40] "GET /cam.mjpg HTTP/1.1" 200 -


stop streaming


127.0.0.1 - - [19/Aug/2018 21:11:42] "GET /cam.mjpg HTTP/1.1" 200 -


In [25]:
try:
    print(here)
except KeyboardInterrupt:
    print("stop server")
    pass
except:
    print
    pass

In [18]:
capture = cv2.VideoCapture(2)
time.sleep(1)
capture.set(cv2.CAP_PROP_FRAME_WIDTH, 800); 
capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 600);
capture.set(cv2.CAP_PROP_SATURATION,0.2);
rc,img = capture.read()
img
capture.release()

NameError: name 'c' is not defined